In [1]:
import requests
import pandas as pd
import os
from datetime import datetime

run_date = datetime.now().strftime('%Y-%m-%d')

input_dir = os.path.join(os.getcwd(), "inputs")
output_dir = os.path.join(os.getcwd(), "outputs")

input_csv = os.path.join(input_dir, "api_result.csv")
output_csv = os.path.join(output_dir, f"quarterly_holdings_as_of_{run_date}.csv")

In [2]:
# only 100 max call per month, loop below counts 50 calls
# replace with your api key registered here https://sec-api.io/docs/n-port-data-api 
api_key = "bc0ce7af2bbc192fbe8f9479428ad787856df154f8fa8ea0054d2fbe8e919835" 
url = f"https://api.sec-api.io/form-nport?token={api_key}"

### Run code below monthly (API free limit refreshes every month), api call result saved to api_result.csv

In [3]:
all_data = pd.DataFrame()

# Loop to fetch data in chunks of 10 (max return per call) until we have 500 results
for i in range(15):
    query_payload = {
        "query": 'genInfo.regCik:"0000810893"', 
        # These are the funds included under cik 0000810893:
        # https://www.sec.gov/Archives/edgar/data/810893/0000810893-19-003090-index.htm
        "from": i * 10, 
        "size": 10,  
        "sort": [{"filedAt": {"order": "desc"}}] 
    }

    response = requests.post(url, json=query_payload)

    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['filings'])
        all_data = pd.concat([all_data, df], ignore_index=True)
    else:
        print(f"Request failed with status code {response.status_code}")
        break

df_clean = all_data[['invstOrSecs', 'filedAt']]
df_clean = df_clean.explode('invstOrSecs')
df_clean = pd.concat([df_clean.drop(['invstOrSecs'], axis=1), df_clean['invstOrSecs'].apply(pd.Series)], axis=1)
df_clean.to_csv(input_csv)

Request failed with status code 429


### Read in saved local csv, clean data to extract TIPS valUSD saved to as_of_run_date csvs

In [4]:
df_clean = pd.read_csv(input_csv)

In [5]:
filtered_df = df_clean[(df_clean['issuerCat'] == 'UST') & (df_clean['title'].str.contains('TSY INFL', case=False))]
filtered_df = filtered_df[['filedAt','title','valUSD']]
filtered_df.loc[:, 'filedAt'] = pd.to_datetime(filtered_df['filedAt'], utc=True).dt.date
filtered_df.loc[:, 'valUSD (MM)'] = (filtered_df['valUSD'] / 1_000_000).astype(float).round(2)

pivot_table = filtered_df.pivot_table(index='title', columns='filedAt', values='valUSD (MM)', aggfunc='sum')
pivot_table = pivot_table.reindex(sorted(pivot_table.columns, reverse=True), axis=1).reset_index()
pivot_table['extracted_date'] = pivot_table['title'].str.extract(r'(\d{2}/\d{2})')[0]
pivot_table['extracted_date'] = pd.to_datetime(pivot_table['extracted_date'], format='%m/%y', errors='coerce')
pivot_table_sorted = pivot_table.sort_values(by='extracted_date').set_index('title').drop('extracted_date', axis=1)
# drop oldest column as it could be incomplete due to query limit
pivot_table_sorted = pivot_table_sorted.iloc[:, :-1] 
pivot_table_sorted.to_csv(output_csv)

### Store history while append data with new runs to merged_quarterly_holdings.csv

In [6]:
csv_files = [f for f in os.listdir(output_dir) if f.endswith('.csv')]
csv_files.sort(key=lambda x: datetime.strptime(x.split('_as_of_')[-1].split('.')[0], '%Y-%m-%d'))

merged_df = pd.DataFrame()

for file in csv_files:
    file_path = os.path.join(output_dir, file)
    df = pd.read_csv(file_path)

    if merged_df.empty:
        merged_df = df
    else:
        # Find columns that are in the new DataFrame but not in the merged DataFrame
        new_columns = df.columns.difference(merged_df.columns)
        merged_df = pd.concat([merged_df, df[new_columns]], axis=1)
        
merged_df = merged_df.set_index('title')
merged_df = merged_df.reindex(sorted(merged_df.columns, reverse=True), axis=1).reset_index()
merged_df.to_excel(os.path.join(os.getcwd(), 'merged_quarterly_holdings.xlsx'), index=False)